<a href="https://colab.research.google.com/github/neutro-jina/study_log/blob/main/%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D%EC%9D%84%20%EC%9C%84%ED%95%9C%20%ED%8C%8C%EC%9D%B4%EC%8D%AC%20%ED%95%9C%20%EC%A1%B0%EA%B0%81/CH06_XOR%EB%AC%B8%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np

# sigmoid 함수
def sigmoid(x):
    return 1 / (1+np.exp(-x))


# 수치미분 함수
def derivative(f, var):    
    
    if var.ndim == 1:  # vector
        
        temp_var = var
        
        delta = 1e-5
        diff_val = np.zeros(var.shape)
        
        for index in range(len(var)):
            
            target_var = float(temp_var[index])
            
            temp_var[index] = target_var + delta            
            
            func_val_plust_delta = f(temp_var)   # x+delta 에 대한 함수 값 계산
        
            temp_var[index] = target_var - delta  
            
            func_val_minus_delta = f(temp_var) # x-delta 에 대한 함수 값 계산
            
            diff_val[index] = ( func_val_plust_delta - func_val_minus_delta ) / (2*delta)
            
            temp_var[index] = target_var
            
        return diff_val
    
        
    elif var.ndim == 2:  # matrix
        
        temp_var = var
        
        delta = 1e-5
        diff_val = np.zeros(var.shape)
    
        rows = var.shape[0]
        columns = var.shape[1]
    
        for row in range(rows):
        
            for column in range(columns):
            
                target_var = float(temp_var[row,column])
            
                temp_var[row,column] = target_var + delta            
                
                func_val_plus_delta = f(temp_var)   # x+delta 에 대한 함수 값 계산
        
                temp_var[row,column] = target_var - delta            
                
                func_val_minus_delta = f(temp_var)   # x-delta 에 대한 함수 값 계산
            
                diff_val[row,column] = ( func_val_plus_delta - func_val_minus_delta ) / (2*delta)
                
                temp_var[row,column] = target_var
                
        return diff_val

In [6]:
# LogicGate Class (논리게이트 클래스 정의)

class LogicGate:
    
    def __init__(self, gate_name, xdata, tdata):  
        #생성자(__init__)에서는 입력 데이터와 정답 데이터, 가중치, 바이어스 그리고 학습률 등을 초기화 해준다.
        self.name = gate_name
        
        # 입력 데이터, 정답 데이터 초기화
        self.xdata = xdata.reshape(4,2)
        self.tdata = tdata.reshape(4,1)
        
        # 가중치 W, 바이어스 b 초기화
        self.W = np.random.rand(2,1)  
        self.b = np.random.rand(1)
                        
        # 학습률 learning rate 초기화
        self.learning_rate = 1e-2
        
    # 손실함수
    def loss_func(self):
        
        delta = 1e-7    # log 무한대 발산 방지
    
        z = np.dot(self.xdata, self.W) + self.b
        y = sigmoid(z)
    
        # cross-entropy 
        return  -np.sum( self.tdata*np.log(y + delta) + (1-self.tdata)*np.log((1 - y)+delta ) )      
    
    # 손실 값 계산
    def loss_val(self):
        
        delta = 1e-7    # log 무한대 발산 방지
    
        z = np.dot(self.xdata, self.W) + self.b
        y = sigmoid(z)
    
        # cross-entropy 
        return  -np.sum( self.tdata*np.log(y + delta) + (1-self.tdata)*np.log((1 - y)+delta ) )

    # 수치미분을 이용하여 손실함수가 최소가 될때 까지 학습하는 함수
    def train(self):
        
        f = lambda x : self.loss_func()
        
        print("Initial loss value = ", self.loss_val())
        
        for step in  range(8001):
            
            self.W -= self.learning_rate * derivative(f, self.W)
    
            self.b -= self.learning_rate * derivative(f, self.b)
    
            if (step % 1000 == 0):
                print("step = ", step, "loss value = ", self.loss_val())
                
                
    # 미래 값 예측 함수
    def predict(self, input_data):
        
        z = np.dot(input_data, self.W) + self.b
        y = sigmoid(z)
    
        if y > 0.5:
            result = 1  # True
        else:
            result = 0  # False
    
        return y, result

In [7]:
xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
tdata = np.array([0, 0, 0, 1])

AND_obj = LogicGate("AND_GATE", xdata, tdata)

AND_obj.train()

Initial loss value =  4.245897735101845
step =  0 loss value =  4.191768776101891
step =  1000 loss value =  0.9808857957924251
step =  2000 loss value =  0.649211771920967
step =  3000 loss value =  0.4852010415255776
step =  4000 loss value =  0.3862896097013882
step =  5000 loss value =  0.3201407563823834
step =  6000 loss value =  0.27287657962657597
step =  7000 loss value =  0.23748554920357617
step =  8000 loss value =  0.21003496163967728


In [8]:
# AND Gate prediction
print(AND_obj.name, "\n")

test_data = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])

for input_data in test_data:
    (sigmoid_val, logical_val) = AND_obj.predict(input_data) 
    print(input_data, " = ", logical_val)  

AND_GATE 

[0 0]  =  0
[0 1]  =  0
[1 0]  =  0
[1 1]  =  1


In [9]:
xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
tdata = np.array([0, 1, 1, 1])

OR_obj = LogicGate("OR_GATE", xdata, tdata)

OR_obj.train() 

Initial loss value =  1.9391670357783466
step =  0 loss value =  1.9290130509709145
step =  1000 loss value =  0.6761407356642997
step =  2000 loss value =  0.4142745585121421
step =  3000 loss value =  0.29461991309786784
step =  4000 loss value =  0.2271114104612265
step =  5000 loss value =  0.1841294031416293
step =  6000 loss value =  0.1545104571076824
step =  7000 loss value =  0.13292784279968095
step =  8000 loss value =  0.11653481749001862


In [10]:
# OR Gate prediction
print(OR_obj.name, "\n")

test_data = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])

for input_data in test_data:
    (sigmoid_val, logical_val) = OR_obj.predict(input_data) 
    print(input_data, " = ", logical_val)  

OR_GATE 

[0 0]  =  0
[0 1]  =  1
[1 0]  =  1
[1 1]  =  1


In [11]:
xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
tdata = np.array([1, 1, 1, 0])

NAND_obj = LogicGate("NAND_GATE", xdata, tdata)

NAND_obj.train()

Initial loss value =  2.831318266849175
step =  0 loss value =  2.8247301024260016
step =  1000 loss value =  1.0521322403865812
step =  2000 loss value =  0.6786250218741112
step =  3000 loss value =  0.5016101890765621
step =  4000 loss value =  0.39677083020647486
step =  5000 loss value =  0.3273996920140586
step =  6000 loss value =  0.278189166637957
step =  7000 loss value =  0.24153442580517476
step =  8000 loss value =  0.21321838032573787


In [12]:
# NAND Gate prediction
print(NAND_obj.name, "\n")

test_data = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])

for input_data in test_data:
    (sigmoid_val, logical_val) = NAND_obj.predict(input_data) 
    print(input_data, " = ", logical_val)  

NAND_GATE 

[0 0]  =  1
[0 1]  =  1
[1 0]  =  1
[1 1]  =  0


In [13]:
xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
tdata = np.array([0, 1, 1, 0])


XOR_obj = LogicGate("XOR_GATE", xdata, tdata)

# XOR Gate 를 보면, 손실함수 값이 2.7 근처에서 더 이상 감소하지 않는것을 볼수 있음
XOR_obj.train()

Initial loss value =  3.061228725342344
step =  0 loss value =  3.053821670970647
step =  1000 loss value =  2.773782092048977
step =  2000 loss value =  2.7726386556202693
step =  3000 loss value =  2.7725900894426765
step =  4000 loss value =  2.77258801490318
step =  5000 loss value =  2.772587926202463
step =  6000 loss value =  2.7725879224093193
step =  7000 loss value =  2.7725879222471077
step =  8000 loss value =  2.772587922240171


In [14]:
# XOR Gate prediction => 예측이 되지 않음
print(XOR_obj.name, "\n")

test_data = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])

for input_data in test_data:
    (sigmoid_val, logical_val) = XOR_obj.predict(input_data) 
    print(input_data, " = ", logical_val)  

XOR_GATE 

[0 0]  =  0
[0 1]  =  0
[1 0]  =  0
[1 1]  =  1


In [15]:
# XOR 을 NAND + OR => AND 조합으로 계산함
input_data = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
print(len(input_data))

s1 = []    # NAND 출력
s2 = []    # OR 출력

new_input_data = []  # AND 입력
final_output = []    # AND 출력

for index in range(len(input_data)):
    
    s1 = NAND_obj.predict(input_data[index])  # NAND 출력
    s2 = OR_obj.predict(input_data[index])    # OR 출력
    
    new_input_data.append(s1[-1])    # AND 입력
    new_input_data.append(s2[-1])    # AND 입력
    
    (sigmoid_val, logical_val) = AND_obj.predict(np.array(new_input_data))
    
    final_output.append(logical_val)    # AND 출력, 즉 XOR 출력    
    new_input_data = []    # AND 입력 초기화


for index in range(len(input_data)):    
    print(input_data[index], " = ", final_output[index])

4
[0 0]  =  0
[0 1]  =  1
[1 0]  =  1
[1 1]  =  0
